In [2]:
import pandas as pd

In [3]:
GRF_F_V_PRO_left = pd.read_csv(r'GRF_F_V_PRO_left.csv')

GRF_F_AP_PRO_left = pd.read_csv(r'GRF_F_AP_PRO_left.csv')

GRF_F_ML_PRO_left = pd.read_csv(r'GRF_F_ML_PRO_left.csv')

GRF_COP_AP_PRO_left = pd.read_csv(r'GRF_COP_AP_PRO_left.csv')

GRF_COP_ML_PRO_left = pd.read_csv(r'GRF_COP_ML_PRO_left.csv')

# Right lower extremity
GRF_F_V_PRO_right = pd.read_csv(r'GRF_F_V_PRO_right.csv')

GRF_F_AP_PRO_right = pd.read_csv(r'GRF_F_AP_PRO_right.csv')

GRF_F_ML_PRO_right = pd.read_csv(r'GRF_F_ML_PRO_right.csv')

GRF_COP_AP_PRO_right = pd.read_csv(r'GRF_COP_AP_PRO_right.csv')

GRF_COP_ML_PRO_right = pd.read_csv(r'GRF_COP_ML_PRO_right.csv')

# Annotations and Metadata
GRF_Annotation = pd.read_csv(r'GRF_metadata.csv')
GRF_Annotation=GRF_Annotation.drop(columns=['SESSION_DATE'])



In [4]:
GRF_Annotation=GRF_Annotation.drop(columns=['HEIGHT','SHOE_SIZE','ORTHOPEDIC_INSOLE','AFFECTED_SIDE'])

GRF_Annotation.head()


,SUBJECT_ID,SESSION_ID,CLASS_LABEL,CLASS_LABEL_DETAILED,SEX,AGE,BODY_WEIGHT,BODY_MASS,SHOD_CONDITION,SPEED,READMISSION,SESSION_TYPE,TRAIN,TRAIN_BALANCED,TEST
0,1,29885,HC,HC,1,18,516.1,52.6,1,2,0,1,0,0,1
1,1,29886,HC,HC,1,18,516.2,52.6,1,1,0,1,0,0,1
2,1,29888,HC,HC,1,18,516.1,52.6,1,3,0,1,0,0,1
3,2,888812549,HC,HC,1,51,752.1,76.7,1,1,0,1,1,0,0
4,2,888812550,HC,HC,1,51,752.4,76.7,1,3,0,1,1,0,0


In [5]:
GRF_Annotation.shape
GRF_Annotation['CLASS_LABEL'].unique()

array(['HC', 'C', 'H', 'K', 'A'], dtype=object)

In [6]:
GRF_Annotation.tail()

,SUBJECT_ID,SESSION_ID,CLASS_LABEL,CLASS_LABEL_DETAILED,SEX,AGE,BODY_WEIGHT,BODY_MASS,SHOD_CONDITION,SPEED,READMISSION,SESSION_TYPE,TRAIN,TRAIN_BALANCED,TEST
8966,2392,15483,A,A_L,1,46,1031.9,105.2,1,2,0,2,0,0,1
8967,2392,15484,A,A_L,1,46,1031.6,105.2,1,2,0,2,0,0,1
8968,2392,15541,A,A_L,1,46,1045.5,106.6,1,2,0,2,0,0,1
8969,2392,15603,A,A_L,1,46,1043.7,106.4,1,2,0,2,0,0,1
8970,2392,15666,A,A_L,1,46,1051.5,107.2,1,2,0,2,0,0,1


In [7]:
GRF_Annotation.isnull().sum()

SUBJECT_ID              0
SESSION_ID              0
CLASS_LABEL             0
CLASS_LABEL_DETAILED    0
SEX                     0
AGE                     0
BODY_WEIGHT             0
BODY_MASS               0
SHOD_CONDITION          0
SPEED                   0
READMISSION             0
SESSION_TYPE            0
TRAIN                   0
TRAIN_BALANCED          0
TEST                    0
dtype: int64

In [19]:
GRF_Annotation.shape

(8971, 15)

In [21]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# Define the test size ratio
test_size_ratio = 0.3
accuracies = []


datasets = [GRF_F_V_PRO_left, GRF_F_AP_PRO_left, GRF_F_ML_PRO_left, GRF_COP_AP_PRO_left, GRF_COP_ML_PRO_left,
            GRF_F_V_PRO_right, GRF_F_AP_PRO_right, GRF_F_ML_PRO_right, GRF_COP_AP_PRO_right, GRF_COP_ML_PRO_right]
for i, dataset in enumerate(datasets):
    #print(f"Dataset {i+1}:")
    
    # Merge dataset with additional_info_df on SUBJECT_ID and SESSION_ID
    dataset_merged = pd.merge(dataset, GRF_Annotation, on=['SUBJECT_ID', 'SESSION_ID'], how='inner')
    
    # Separate features and target variable
    X = dataset_merged.drop(columns=['CLASS_LABEL', 'CLASS_LABEL_DETAILED', 'SUBJECT_ID', 'SESSION_ID'])  # Features
    y = dataset_merged['CLASS_LABEL']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size_ratio, random_state=42)
    
    # Define the parameters for KNN
    n_neighbors = 2
    weights = 'uniform'
    metric = 'minkowski'
    p = 2  # p = 2 corresponds to Euclidean distance
    
    # Initialize KNN classifier with specified parameters
    knn = KNeighborsClassifier(n_neighbors=n_neighbors, weights=weights, metric=metric, p=p)
    
    # Train the classifier
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    #print(f"Accuracy: {accuracy}")
    
    # Append accuracy to the list
    accuracies.append(accuracy)
    #adjusted_predictions = ['Healthy' if label == 'HC' else 'Not Healthy' for label in y_pred]

# Calculate the overall accuracy
overall_accuracy = sum(accuracies) / len(accuracies)
print(f"Overall Accuracy: {overall_accuracy}")
    


Overall Accuracy: 0.8291285211267605


In [16]:
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)



Shape of X_train: (53012, 113)
Shape of X_test: (22720, 113)
Shape of y_train: (53012,)
Shape of y_test: (22720,)


In [22]:
#y_pred = knn.predict(X_test)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Evaluate the predictions
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.8207306338028169
Precision: 0.8358058939427724
Recall: 0.8207306338028169
F1-score: 0.8192257086480825
Confusion Matrix:
[[6106  135  107   15   88]
 [ 666 3308   71    9   58]
 [ 573  329 2877   15   72]
 [  30   37   29 2204    1]
 [ 803  507  425  103 4152]]


In [24]:
GRF_Annotation['CLASS_LABEL'].unique()


array(['HC', 'C', 'H', 'K', 'A'], dtype=object)

In [25]:
# Print the shapes of y_pred and y_test
print("Shape of y_pred:", y_pred.shape)
print("Shape of y_test:", y_test.shape)

# Print the values of y_pred and y_test
print("Values of y_pred:", y_pred)
print("Values of y_test:", y_test)


Shape of y_pred: (22720,)
Shape of y_test: (22720,)
Values of y_pred: ['H' 'A' 'A' ... 'A' 'H' 'A']
Values of y_test: 56104    K
22112    H
54148    A
16754    K
67428    K
        ..
45443    K
37239    C
23819    A
50280    H
12613    A
Name: CLASS_LABEL, Length: 22720, dtype: object


In [26]:
# Reset index of y_test
y_test_reset = y_test.reset_index(drop=True)

# Compare the predicted outputs with the actual labels
for i in range(len(y_pred)):
    print("Predicted:", y_pred[i], "\tActual:", y_test_reset[i])


Predicted: H 	Actual: K
Predicted: A 	Actual: H
Predicted: A 	Actual: A
Predicted: K 	Actual: K
Predicted: C 	Actual: K
Predicted: H 	Actual: H
Predicted: HC 	Actual: HC
Predicted: A 	Actual: A
Predicted: A 	Actual: A
Predicted: C 	Actual: C
Predicted: K 	Actual: K
Predicted: C 	Actual: C
Predicted: K 	Actual: K
Predicted: H 	Actual: H
Predicted: A 	Actual: A
Predicted: A 	Actual: A
Predicted: K 	Actual: K
Predicted: A 	Actual: A
Predicted: A 	Actual: K
Predicted: C 	Actual: C
Predicted: C 	Actual: C
Predicted: K 	Actual: K
Predicted: K 	Actual: K
Predicted: A 	Actual: A
Predicted: HC 	Actual: HC
Predicted: C 	Actual: C
Predicted: H 	Actual: H
Predicted: HC 	Actual: HC
Predicted: A 	Actual: A
Predicted: C 	Actual: C
Predicted: K 	Actual: K
Predicted: H 	Actual: H
Predicted: K 	Actual: K
Predicted: HC 	Actual: K
Predicted: A 	Actual: H
Predicted: C 	Actual: C
Predicted: C 	Actual: C
Predicted: K 	Actual: K
Predicted: HC 	Actual: HC
Predicted: K 	Actual: K
Predicted: C 	Actual: K
Predict

In [27]:
label_mapping = {'HC': 'Healthy', 'H': 'Hip', 'K': 'Knee', 'A': 'Ankle', 'C': 'Calcaneus'}

# Adjust predictions by mapping them back to their original values
adjusted_predictions = [label_mapping[label] for label in y_pred]

# Print adjusted predictions
print("Adjusted Predictions:")
for actual, predicted in zip(y_test, adjusted_predictions):
    print(f"Actual: {actual}\tPredicted: {predicted}")

Adjusted Predictions:
Actual: K	Predicted: Hip
Actual: H	Predicted: Ankle
Actual: A	Predicted: Ankle
Actual: K	Predicted: Knee
Actual: K	Predicted: Calcaneus
Actual: H	Predicted: Hip
Actual: HC	Predicted: Healthy
Actual: A	Predicted: Ankle
Actual: A	Predicted: Ankle
Actual: C	Predicted: Calcaneus
Actual: K	Predicted: Knee
Actual: C	Predicted: Calcaneus
Actual: K	Predicted: Knee
Actual: H	Predicted: Hip
Actual: A	Predicted: Ankle
Actual: A	Predicted: Ankle
Actual: K	Predicted: Knee
Actual: A	Predicted: Ankle
Actual: K	Predicted: Ankle
Actual: C	Predicted: Calcaneus
Actual: C	Predicted: Calcaneus
Actual: K	Predicted: Knee
Actual: K	Predicted: Knee
Actual: A	Predicted: Ankle
Actual: HC	Predicted: Healthy
Actual: C	Predicted: Calcaneus
Actual: H	Predicted: Hip
Actual: HC	Predicted: Healthy
Actual: A	Predicted: Ankle
Actual: C	Predicted: Calcaneus
Actual: K	Predicted: Knee
Actual: H	Predicted: Hip
Actual: K	Predicted: Knee
Actual: K	Predicted: Healthy
Actual: H	Predicted: Ankle
Actual: C	Pr

In [28]:
from sklearn.model_selection import cross_val_score



cv_scores = cross_val_score(knn, X_train, y_train, cv=5)


# Print the cross-validation scores
print("Cross-validation scores:", cv_scores)

# Calculate the mean cross-validation score
mean_cv_score = cv_scores.mean()
print("Mean cross-validation score:", mean_cv_score)

Cross-validation scores: [0.76478355 0.76865038 0.778721   0.77815507 0.77740049]
Mean cross-validation score: 0.7735420970772724
